In [ ]:
%load_ext autoreload
%autoreload 2

figsize=(14, 4)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils import file, plot, data, stat
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
pd.set_option('display.max_columns', None)
data_folder = os.path.join('../../..', 'data/prod')
file_name_collection = "Collection_data.csv"
file_path_collection = os.path.join(data_folder, file_name_collection)
file_name_konto = "konto_data_trimmed.csv"
file_path_konto = os.path.join(data_folder, file_name_konto)

In [ ]:
konto_frame, collection_frame = file.load_konto_data(
    file_path_konto
), file.load_collection_data(file_path_collection)
konto_frame = data.add_relevant_collectionid_to_konto_frame(
    konto_frame, collection_frame
)
konto_frame = data.reorder_column(konto_frame, "Collectionid", 3)
konto_frame = data.reorder_column(konto_frame, "CollectionOpenedAfter202309", 4)
konto_frame = data.reorder_column(konto_frame, "CollectionHasClosedDate", 5)

In [ ]:
descs = sorted(konto_frame["GeneralStatusDesc"].unique())
descs

In [ ]:
for desc in descs:
    ids = konto_frame.loc[konto_frame["GeneralStatusDesc"] == desc, "PersonId"].unique()
    print(f"{desc}: {len(ids)} personer")
    print(f"Første 10: {ids[:10]}")
    print("-" * 40)

In [ ]:
# Get sets of IDs for each condition
avsluttet_ids = set(
    konto_frame.loc[
        konto_frame["GeneralStatusDesc"] == "Purring", "PersonId"
    ]
)
under_avslutning_ids = set(
    konto_frame.loc[
        konto_frame["GeneralStatusDesc"] == "Faktura forfalt", "PersonId"
    ]
)

# Get IDs that appear in both
both_ids = avsluttet_ids & under_avslutning_ids

print(f"{len(both_ids)} PersonId(s) found with both statuses.")
print(list(both_ids)[:10])  # show first 10

### Collectiondata
- `YearMonth`: År og måned i formatet ÅÅÅÅMM
- `Collectionid`: ID for inkassosaken
- `PersonId`: ID for person
- `Registreringsmaaned`: registreringsperiode for når sak ble sendt til inkasso i formatet ÅÅÅÅMM
- `AccountId`: ID for konto
- `Productname`: Produktnavn
- `CollectionOpenedDate`: Dato for når sak ble sendt til inkasso
- `CollectionClosedDate`: Dato for når inkassosak ble lukket
- `BalanceSentAmt`: Balanse som ble sendt til inkasso (Hovedstol)
- `PaymentToKredittbankenAmt`: Innbetalinger til Kredittbanken direkte  i perioden
- `CumulativePaymentToKredittbankenAmt`: Sum innbetalt til Kredittbanken direkte på sak totalt 
- `PaymentToCollectionAmt`: Innbetalinger via inkassoselskapet i perioden
- `CumulativePaymentToCollectionAmt`: Sum innbetalinger gjort til inkassoselskapet totalt
- `LossAmt`: Tapsført i perioden
- `CumulativeLossAmt`: Totalt tapsførst på sak
- `MonthInDCA`: Antall måneder sak har ligget hos inkasso
- `MonthsInZCOV`: Antall måneder sak har ligget til overvåk (tapsført)

### Kontodata
- `YearMonth`: År og måned i formatet ÅÅÅÅMM
- `PeriodId`: Siste dag i gitt periode i formatet ÅÅÅÅMMDD
- `PersonId`: ID for person
- `AccountId`: ID for konto
- `ProductId`: ID for produkt
- `DistributorId`: ID for distributør
- `AgeGroup`: Aldersgruppe
- `Gender`: Kjønn
- `Kommunenavn`: Kommunenavn
- `GeneralStatusDesc`: Beskrivelse av statuskode på konto
- `MonthsSinceAccountCreatedNum`: Antall måneder siden konto ble opprettet
- `BalanceAmt`: Balanse
- `InterestEarningLendingAmt`: Rentebærende balanse
- `CreditLimitAmt`: Kredittgrense
- `CreditLimitInceaseFlag`: 1 dersom det i perioden har blitt foretatt en kredittøkning
- `CollectionFlag`: 1 dersom kontoen ligger til inkasso
- `ClosedAccountFlag`: 1 dersom konto er avsluttet
- `TurnoverAmt`: Beløp varekjøp
- `TurnoverNum`: Antall varekjøpstransaksjoner
- `TurnoverDomAmt`: Beløp varekjøp, innenlands
- `TurnoverDomNum`: Antall varekjøpstransaksjoner, innenlands
- `TurnoverIntAmt`: Beløp varekjøp, utenlands
- `TurnoverIntNum`: Antall varekjøpstransaksjoner, utenlands
- `FundtransferAmt`: Beløp overføring fra kort til konto
- `FundtransferNum`: Antall overføringer fra kort til konto
- `CashAtmAmt`: Kontanter minibank, beløp
- `CashAtmNum`: Kontanter minibank, antall
- `CashCounterAmt`: Kontanter skranke, beløp
- `CashCounterNum`: Kontanter skranke, antall
- `OverdueAmt`: Beløp som er forfalt
- `StatementClosingBalanceAmt`: Balanse ved fakturakjøring
- `StatementEffectivePaymentsAmt`: Innbetalinger mot fakturert beløp. Trekker fra det som eventuelt er betalt over skyldig beløp slik at innbetaling max kan være lik utestående
- `StatementMinimumToPayAmt`: Minimumsbeløp på faktura
- `PaymentOverDueFlag`: Betaling forfalt
- `FullpayerFlag`: 1 dersom det ikke påløper rente på faktura som forfalt  i måneden (hele faktura er betalt)
- `RevolvingFlag`: 1 dersom det påløper renter på fakturra som forfalt i denne måneden (ikke betalt hele utestående)
- `MonthsInDCANum`: Antall måned til inkasso (Debt Collection Agency)
- `MonthsInZCOVNum`: Antall måneder i overvåk
- `ActiveFlag`: 1 dersom en aktiv konto
- `CardUsedFlag`: 1 dersom kortet er brukt
- `TransactionFlag`: 1 dersom det har vært transaksjoner på konto
- `SumL12, L3 og L1`: Summer for de siste 12, 3 og 1 månedene for relevante kategorier

### IDs

In [ ]:
id_col = "PersonId"
n = 100
first_n_ids = konto_frame[id_col].unique()[:n]
first_n_ids

In [ ]:
import pandas as pd
from IPython.display import display, HTML

id = 245991
kont = konto_frame[konto_frame["PersonId"] == id].copy()
coll = collection_frame[collection_frame["PersonId"] == id].copy()

ref_date = pd.Timestamp("2023-09-01")

# Add "Måneder siden start"
for df in (kont, coll):
    df["Måneder siden start"] = (df["YearMonth"].dt.year - ref_date.year) * 12 + (
        df["YearMonth"].dt.month - ref_date.month
    )

# Ensure both have same range of months
all_months = pd.Series(range(0, 25), name="Måneder siden start")

# Reindex each to include all months, filling missing ones with NaN
kont = all_months.to_frame().merge(kont, on="Måneder siden start", how="left")
coll = all_months.to_frame().merge(coll, on="Måneder siden start", how="left")

coll["Collectionid"] = coll["Collectionid"].astype("Int64")
coll["PersonId"] = coll["PersonId"].astype("Int64")
coll["AccountId"] = coll["AccountId"].astype("Int64")
coll["MonthInDCA"] = coll["MonthInDCA"].astype("Int64")

kont.rename(columns={"StatementMinimumToPayAmt": "Minimumsbeløp på faktura"}, inplace=True)
kont.rename(
    columns={"StatementEffectivePaymentsAmt": "Innbetalinger mot fakturert beløp"},
    inplace=True,
)
kont.rename(
    columns={"StatementClosingBalanceAmt": "Balanse ved fakturakjøring"}, inplace=True
)
kont.rename(columns={"OverdueAmt": "Beløp som er forfalt"}, inplace=True)
kont.rename(columns={"InterestEarningLendingAmt": "Rentebærende balanse"}, inplace=True)
kont.rename(columns={"BalanceAmt": "Balanse"}, inplace=True)
kont.rename(columns={"TurnoverAmt": "Betalinger sum"}, inplace=True)
kont.rename(columns={"FundtransferAmt": "Uttak fra kort til konto"}, inplace=True)
kont.rename(columns={"CashAtmAmt": "Uttak minibank"}, inplace=True)
kont.rename(columns={"CashCounterAmt": "Uttak butikkskranke"}, inplace=True)

cols_to_drop = [
    "TurnoverNum",
    "TurnoverDomNum",
    "TurnoverIntNum",
    "FundtransferNum",
    "CashAtmNum",
    "CashCounterNum",
]
kont.drop(columns=cols_to_drop, inplace=True)

# --- Display side-by-side with consistent row height ---
html = f"""
<style>
.dataframe td, .dataframe th {{
    line-height: 1.2em;
    height: 2.4em;
    vertical-align: middle;
    white-space: nowrap;
}}
</style>

<div style="display: flex; justify-content: space-between; gap: 10px;">
  <div style="width: 49%; overflow-x: auto; border: 1px solid #ccc; padding: 5px;">
    {kont.to_html(index=False, na_rep='')}
  </div>
  <div style="width: 49%; overflow-x: auto; border: 1px solid #ccc; padding: 5px;">
    {coll.to_html(index=False, na_rep='')}
  </div>
</div> 
"""

display(HTML(html))